 # 1. Setup

In [1]:
from caremel_predict.interfaces.data_interactor import DataInteractor
import collections
import numpy as np
import pandas as pd
import sqlalchemy
import os
# Import date and timedelta class
# from datetime module
from datetime import date
from datetime import timedelta

#request 
import requests
di = DataInteractor()

#Functions
def _add_in_dict_ds(row):
    try:
        ds_dict[row['member_internal_code']] = {
            **ds_dict[row['member_internal_code']],
            **{
                row['cid']: 1
            }
        }
    except:
        ds_dict[row['member_internal_code']] = {row['cid']: 1}
#
def _add_in_dict_fp(row):
    if row['question_text'] in QUESTIONS:
        try:
            fp_dict[row['member_internal_code']] = {
                **fp_dict[row['member_internal_code']],
                **{
                    row['question_text']: row['answer_text']
                }
            }
        except:
            fp_dict[row['member_internal_code']] = {
                row['question_text']: row['answer_text']
            }
#
def _request_first_layer(input):
    r = requests.post(
        "https://sm-gateway.datalake.alice.tools/caramelo/first_layer",
        json=input)
    if r.status_code == 200:
        return r.json()
    else:
        print("Request failed")
        return None

#
def _request_second_layer(input):
    r = requests.post(
        "https://sm-gateway.datalake.alice.tools/caramelo/second_layer",
        json=input)
    if r.status_code == 200:
        return r.json()
    else:
        print("Request failed")
        return None

# Questions from questionnaire to be considered
QUESTIONS = ['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.',
                'Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.',
                'Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.',
                'Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.',
                'Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.',
                'Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.',
                'Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.',
                'O que te incomoda em relação à sua saúde?',
                'Qual a sua altura? (em cm)',
                'Qual o seu peso atual? (em kg)',
                'Você fuma ou já fumou?',
                'Você usa drogas recreativas (como maconha ou outras drogas sintéticas)?',    
                'Se sentiu para baixo, deprimido(a) ou sem perspectiva?',
                'Teve pouco interesse ou pouco prazer em fazer as coisas?',
                'Não ser capaz de impedir ou controlar as preocupações',
                'Sentir-se nervoso(a), ansioso(a) ou muito tenso(a)',
                'Em uma escala de 0 a 10, que nota você dá à sua saúde hoje?']

In [2]:
onboarding_df = di.redshift.run_sql_query(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/queries/onboarding_df.sql')
df_ds = di.redshift.run_sql_query(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/queries/df_ds.sql')
df_fp = di.redshift.run_sql_query(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/queries/df_fp.sql')

/Users/luiz.superti/Library/Python/3.9/lib/python/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/luiz.superti/Library/Python/3.9/lib/python/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/luiz.superti/Library/Python/3.9/lib/python/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## 3. Transform

In [3]:
#list of cids codes
df_ds['cid'] = df_ds['cid'].apply(lambda x: x[0:3])
ds_dict = collections.defaultdict()

df_ds.apply(lambda row: _add_in_dict_ds(row), axis=1)
df_ds_row = pd.DataFrame.from_dict(ds_dict, orient='index').reset_index(drop=False).rename(columns={'index': 'member_internal_code'})
df_ds_row = df_ds_row.merge(df_ds)

df_ds_row.fillna(0, inplace=True)
df_ds_row.drop(columns='cid', inplace=True)
df_ds_row['is_risk_or_chronic_dis'] = np.where(
(df_ds_row['E66'] == 1) | (df_ds_row['N63'] == 1) |
(df_ds_row['I10'] == 1) | (df_ds_row['E05'] == 1) |
(df_ds_row['E11'] == 1) | (df_ds_row['B24'] == 1) |
(df_ds_row['F84'] == 1) | (df_ds_row['M05'] == 1) |
(df_ds_row['Q90'] == 1) | (df_ds_row['I64'] == 1) |
(df_ds_row['I50'] == 1) | (df_ds_row['N18'] == 1) |
(df_ds_row['Z95'] == 1), 1, 0)

df_ds_row.drop_duplicates(inplace=True)


# questionnaires
fp_dict = collections.defaultdict()
df_fp.apply(lambda row: _add_in_dict_fp(row), axis=1)
df_fp_row = pd.DataFrame.from_dict(fp_dict, orient='index').reset_index(drop=False).rename(
                                                             columns={'index': 'member_internal_code'})
df_fp_row.fillna(-1, inplace=True)


#gad and phq
df_fp_row['phq_2_w'] = df_fp_row['Se sentiu para baixo, deprimido(a) ou sem perspectiva?'].astype(float) + df_fp_row['Teve pouco interesse ou pouco prazer em fazer as coisas?'].astype(float)
df_fp_row['phq_2_w'] = df_fp_row['phq_2_w'].fillna(-1)
df_fp_row.loc[df_fp_row['phq_2_w']<0,'phq_2_w'] = -1

df_fp_row['gad_2_w'] = df_fp_row['Não ser capaz de impedir ou controlar as preocupações'].astype(float) + df_fp_row['Sentir-se nervoso(a), ansioso(a) ou muito tenso(a)'].astype(float)
df_fp_row['gad_2_w'] = df_fp_row['gad_2_w'].fillna(-1)
df_fp_row.loc[df_fp_row['gad_2_w']<0,'gad_2_w'] = -1

#euroqol
df_fp_row['euroqol_w'] = df_fp_row['Em uma escala de 0 a 10, que nota você dá à sua saúde hoje?'].astype(float)/10
df_fp_row['euroqol_w'] = df_fp_row['euroqol_w'].fillna(-1)


#blood pressure, cancer, heart attack and diabetes questions
df_fp_row['has_ancestry_blood_pressure'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_blood_pressure'] = 0

#heart attack
df_fp_row['has_ancestry_heart_attack0'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_heart_attack0'] = 0
df_fp_row['has_ancestry_heart_attack1'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_heart_attack1'] = 0

##cancers
df_fp_row['has_ancestry_cancer0'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_cancer0'] = 0
df_fp_row['has_ancestry_cancer1'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_cancer1'] = 0
df_fp_row['has_ancestry_cancer2'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_cancer2'] = 0

#diabetes
df_fp_row['has_ancestry_diabetes'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.']=='Não', 'has_ancestry_diabetes'] = 0

df_fp_row['has_ancestry_heart_attack'] = np.where((df_fp_row['has_ancestry_heart_attack0']==1)|(df_fp_row['has_ancestry_heart_attack1']==1),1,-1)
df_fp_row.loc[(df_fp_row['has_ancestry_heart_attack0']==0)&(df_fp_row['has_ancestry_heart_attack1']==0),'has_ancestry_heart_attack'] = 0

df_fp_row['has_ancestry_cancer'] = np.where((df_fp_row['has_ancestry_cancer0']==1)|(df_fp_row['has_ancestry_cancer1']==1)|(df_fp_row['has_ancestry_cancer2']==1),1,-1)
df_fp_row.loc[(df_fp_row['has_ancestry_cancer0']==0)&(df_fp_row['has_ancestry_cancer1']==0)&(df_fp_row['has_ancestry_cancer2']==0),'has_ancestry_cancer'] = 0



# has some medical condition that wants to treat
df_fp_row['has_incomodo_para_tratar_len'] = df_fp_row['O que te incomoda em relação à sua saúde?'].apply(lambda x: len(str(x)))
df_fp_row['v1'] = df_fp_row['O que te incomoda em relação à sua saúde?'].apply(lambda x:str.lower(str(x)))
df_fp_row['has_incomodo_para_tratar'] = np.where((df_fp_row['has_incomodo_para_tratar_len']>3) & (
                                                    df_fp_row['v1']!='nada') &  (
                                                    df_fp_row['v1']!='não sei') & (
                                                    df_fp_row['v1']!='nada em específico') & (
                                                    df_fp_row['v1']!='por enquanto nada') & (
                                                    df_fp_row['v1']!='no momento nada') & (
                                                    df_fp_row['v1']!='nada em particular') & (
                                                    df_fp_row['v1']!='nada.') & (
                                                    df_fp_row['v1']!='ainda nada') & (
                                                    df_fp_row['v1']!='por enquanto nada.') & (
                                                    df_fp_row['v1']!='por enquanto, nada.') & (
                                                    df_fp_row['v1']!='nada até o momento') & (
                                                    df_fp_row['v1']!='Ainda nada') & (
                                                    df_fp_row['v1']!='No momento nada :)'
                                                    ),1,0)
df_fp_row.loc[df_fp_row['O que te incomoda em relação à sua saúde?'] == 'null', 'has_incomodo_para_tratar'] = -1
df_fp_row.loc[df_fp_row['v1'] == '-1', 'has_incomodo_para_tratar'] = -1

#smoke
# Fumo
df_fp_row['smoke'] = np.where(df_fp_row['Você fuma ou já fumou?']=='10',0,1)
df_fp_row.loc[df_fp_row['Você fuma ou já fumou?'] == '-1', ['smoke']] = -1

# drogas
df_fp_row['v1'] = df_fp_row['Você usa drogas recreativas (como maconha ou outras drogas sintéticas)?'].apply(lambda x: str(x).partition('@')[0])
df_fp_row['drug'] = np.where(df_fp_row['v1']=='Não',0,1)
df_fp_row.loc[df_fp_row['v1'] == "-1", ['drug']] = -1
df_fp_row.loc[df_fp_row['v1'] == "null", ['drug']] = -1

#imc
df_fp_row['imc'] = df_fp_row['Qual o seu peso atual? (em kg)'].apply(lambda x: float(x)) / (df_fp_row['Qual a sua altura? (em cm)'].apply(lambda x:float(x))/100)**2
df_fp_row['imc'] = df_fp_row['imc'].fillna(-1)
df_fp_row.loc[df_fp_row['imc']<0,'imc'] = -1
df_fp_row['imc'] = df_fp_row['imc'].apply(lambda x: round(x,2))


#static info
df0 = df_fp[['member_internal_code', 'member_sex']]
df0.drop_duplicates(inplace=True)


/var/folders/cg/wkjm197x02l9tmhwc_l_18km0000gr/T/ipykernel_11099/808925311.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df0.drop_duplicates(inplace=True)


In [4]:
ml_df0 = df_fp_row[['member_internal_code',
                    'has_ancestry_diabetes',
                    'gad_2_w',
                    'phq_2_w',
                    'euroqol_w',
                    'has_ancestry_blood_pressure',
                    'has_ancestry_heart_attack',
                    'has_ancestry_cancer',
                    'has_incomodo_para_tratar',
                    'smoke',
                    'drug',
                    'imc']].merge(df_ds_row[['member_internal_code', 'is_risk_or_chronic_dis']],how = 'left', on = 'member_internal_code')

In [5]:
static_info_df = onboarding_df[['member_internal_code',
                                'onboarding_risk_value',
                                 'age',
                                 'status_onboarding_curated',
                                 'onboarding_form_ended_at',
                                 'follow_up_expected']].merge(df0, how = 'left', left_on = 'member_internal_code', right_on = 'member_internal_code')
static_info_df['is_female'] = np.where(static_info_df['member_sex']=='FEMALE',1,0)
static_info_df.drop(columns=['member_sex'],inplace=True)

ml_df = static_info_df.merge(ml_df0, how  = 'left', on = 'member_internal_code')
#add variables
ml_df['is_risk_or_chronic_dis'] = ml_df['is_risk_or_chronic_dis'].fillna(0)
ml_df['has_ancestry_rheum'] = -1

ml_df.fillna(-1,inplace=True)

ml_df.rename(columns = {'onboarding_risk_value': 'ds_mat_score',
                        'imc': 'imc_w',
                        'has_incomodo_para_tratar': 'tem_incomodo_p_tratar'}, inplace = True)


In [6]:
ml_df

,member_internal_code,ds_mat_score,age,status_onboarding_curated,onboarding_form_ended_at,follow_up_expected,is_female,has_ancestry_diabetes,gad_2_w,phq_2_w,euroqol_w,has_ancestry_blood_pressure,has_ancestry_heart_attack,has_ancestry_cancer,tem_incomodo_p_tratar,smoke,drug,imc_w,is_risk_or_chronic_dis,has_ancestry_rheum
0,NC1ABF9,0.0,35,Não Finalizou,-1,Jornada Bem-Estar,0,1.0,-1.0,-1.0,0.7,0.0,0.0,0.0,1.0,1.0,-1.0,23.29,0.0,-1
1,NC1PZSD,0.0,30,Indefinido,-1,Jornada Bem-Estar,1,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.00,0.0,-1
2,NC1KSEF,1.0,50,Finalizou,2022-09-20 04:49:03.768330+00:00,Consulta 20 min,1,1.0,2.0,1.0,0.9,1.0,1.0,0.0,1.0,0.0,1.0,21.51,1.0,-1
3,NC14826,2.0,23,Finalizou,2022-09-19 14:51:47.443841+00:00,Consulta 20 min,1,0.0,5.0,4.0,0.7,1.0,1.0,0.0,1.0,0.0,0.0,29.69,0.0,-1
4,NC15X0O,1.0,32,Finalizou,2022-09-20 15:35:57.466999+00:00,Consulta 20 min,0,1.0,4.0,2.0,0.8,1.0,0.0,1.0,1.0,1.0,0.0,25.95,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,NC1DWFL,1.0,21,Finalizou,2022-11-09 14:14:03.137915+00:00,Consulta 20 min,1,1.0,4.0,3.0,0.7,0.0,0.0,-1.0,1.0,0.0,1.0,17.31,0.0,-1
956,NC1LSXN,3.0,26,Finalizou,2022-11-09 22:05:27.746527+00:00,Consulta 20 min,1,0.0,3.0,2.0,0.8,0.0,0.0,0.0,1.0,0.0,0.0,24.61,0.0,-1
957,NC1EA3U,0.0,52,Finalizou,2022-11-10 00:46:31.406308+00:00,Consulta 20 min,1,1.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,21.63,0.0,-1
958,NC1PC4U,0.0,1,Criança,-1,Criança,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,0.0,-1


***

In [7]:
ml_df['caremel_pipe_input'] = np.where(ml_df['follow_up_expected']=='Consulta 20 min',1,-1)
ml_df.loc[ml_df['follow_up_expected']=='Jornada Bem-Estar', 'caremel_pipe_input'] = 1

ml_df['member_valid_for_caremel'] = np.where(ml_df['status_onboarding_curated']=='Finalizou',1,0)

In [8]:
#ml_df['onboard_form_yesterday'] = np.where(ml_df['onboarding_form_ended_at'].apply(lambda x: pd.to_datetime(x, errors = 'coerce').date() ) == date.today() - timedelta(days=1),1,0)

In [9]:
#caremel_input = ml_df[(ml_df['caremel_pipe_input']!=-1) & (ml_df['member_valid_for_caremel']==1) & (ml_df['onboard_form_yesterday']==1)] 
caremel_input = ml_df[(ml_df['caremel_pipe_input']!=-1) & (ml_df['member_valid_for_caremel']==1)]
caremel_input = caremel_input.drop(columns = ['status_onboarding_curated', 'onboarding_form_ended_at', 'follow_up_expected', 'member_valid_for_caremel'])

## 5. Request & Prediction

In [10]:
from bs4 import BeautifulSoup as bs4
import datetime
from tqdm import tqdm


first_layer_times = []
second_layer_times = []
for index, row in tqdm(caremel_input.iterrows(), total=caremel_input.shape[0]):
    if row['caremel_pipe_input'] == 1:
        aux_time = datetime.datetime.now()
        aux_first_risk = _request_first_layer(row.to_dict())
        first_layer_times.append(
            (datetime.datetime.now() - aux_time).total_seconds())
        caremel_input.loc[index, 'first_layer_risk'] = aux_first_risk['risk']


100%|██████████| 707/707 [10:00<00:00,  1.18it/s]


In [11]:
caremel_input['follow_up_caremel'] = np.where(caremel_input['first_layer_risk'] == 'NO_RISK', 'Jornada Bem-Estar', 'Consulta 20 min')
#caremel_input.loc[caremel_input['second_layer_risk'] =='MEDIUM_RISK', 'follow_up_caremel'] = 'Consulta 20 min'
caremel_input['health_plan_task_caremel'] = np.where(caremel_input['first_layer_risk'] == 'NO_RISK', 'PDA Pilar', 'Med - Consulta Longitudinal')
#caremel_input.loc[caremel_input['second_layer_risk'] =='MEDIUM_RISK', 'health_plan_task_caremel'] = 'Med - Consulta Longitudinal'
caremel_input['appointment_orientations_caremel'] = np.where((caremel_input['first_layer_risk'] == 'LOW_RISK'),'De acordo com a AI da Wanda, o risco pode ser um pouco maior do que o apontado na DS/Videocall','')  # type: ignore
#caremel_input.loc[caremel_input['second_layer_risk'] =='MEDIUM_RISK', 'appointment_orientations_caremel'] = 'De acordo com a AI da Wanda, o risco pode ser um pouco maior do que o apontado na DS/Videocall'

In [12]:
new_onboarding_df = onboarding_df.merge(caremel_input[['member_internal_code', 'follow_up_caremel','health_plan_task_caremel','appointment_orientations_caremel']], how = 'left', on = 'member_internal_code')

#FINAL FOLLOW-UP: O ANTERIOR + MUDANCAS TRAZIDAS PELO CAREMEL
new_onboarding_df['final_follow_up'] = new_onboarding_df['follow_up_expected']
new_onboarding_df.loc[(onboarding_df['onboarding_risk'] =='NO_RISK') & (new_onboarding_df['follow_up_caremel'] =='Consulta 20 min') , 'final_follow_up'] = 'Consulta 20 min'
new_onboarding_df.loc[(onboarding_df['onboarding_risk'] =='LOW_RISK') & (new_onboarding_df['follow_up_caremel'] =='Jornada Bem-Estar') , 'final_follow_up'] = 'Jornada Bem-Estar'

#FINAL HEALTH PLAN-TRASK: O ANTERIOR + MUDANCAS TRAZIDAS PELO CAREMEL
new_onboarding_df['final_health_plan_task'] = new_onboarding_df['health_plan_task_expected']
new_onboarding_df.loc[(new_onboarding_df['onboarding_risk'] =='NO_RISK') & (new_onboarding_df['health_plan_task_caremel'] =='Med - Consulta Longitudinal') , 'final_health_plan_task'] = 'Med - Consulta Longitudinal'
new_onboarding_df.loc[(new_onboarding_df['onboarding_risk'] =='LOW_RISK') & (new_onboarding_df['health_plan_task_caremel'] =='PDA Pilar') , 'final_health_plan_task'] = 'PDA Pilar'



#FINAL appointment orientations: O ANTERIOR + MUDANCAS TRAZIDAS PELO CAREMEL
new_onboarding_df['final_appointment_orientations'] = new_onboarding_df['appointment_orientations']
new_onboarding_df.loc[(new_onboarding_df['onboarding_risk'] =='NO_RISK') & (new_onboarding_df['appointment_orientations_caremel'] =='De acordo com a AI da Wanda, o risco pode ser um pouco maior do que o apontado na DS/Videocall') , 'final_appointment_orientations'] = 'De acordo com a AI da Wanda, o risco pode ser um pouco maior do que o apontado na DS/Videocall'
#onboarding_df.loc[(onboarding_df['onboarding_risk'] =='LOW_RISK') & (onboarding_df['appointment_orientations_caremel'] =='PDA Pilar') , 'final_follow_up'] = 'PDA Pilar'

#final_onboarding_df_skew = onboarding_df

In [13]:
new_onboarding_df.columns

Index(['member_internal_code', 'age', 'person_id', 'membership_phi_sk',
       'active_membership_since', 'membership_status', 'membership_count',
       'total_membership_count', 'canceled_at', 'acquisition_type',
       'status_onboarding_updated_at', 'status_onboarding',
       'onboarding_form_ended_at', 'status_onboarding_form',
       'status_onboarding_curated', 'onboarding_risk', 'onboarding_risk_value',
       'onboarding_risk_added_at', 'follow_up_expected',
       'health_plan_task_expected', 'appointment_orientations',
       'appointment_demands', 'flag_score_mental', 'follow_up_caremel',
       'health_plan_task_caremel', 'appointment_orientations_caremel',
       'final_follow_up', 'final_health_plan_task',
       'final_appointment_orientations'],
      dtype='object')

In [16]:
new_onboarding_df[['follow_up_expected','follow_up_caremel']].value_counts()

follow_up_expected  follow_up_caremel
Consulta 20 min     Consulta 20 min      306
                    Jornada Bem-Estar    207
Jornada Bem-Estar   Jornada Bem-Estar    156
                    Consulta 20 min       38
dtype: int64

#### append to old dataframe

In [ ]:
#onboard_ds = onboard_ds.merge(onboarding_df_skew, how = 'left', on = 'member_internal_code')
#onboard_ds['final_follow_up'] = np.where(onboard_ds['final_follow_up_y'].isna(), onboard_ds['final_follow_up_x'], onboard_ds['final_follow_up_y'])
#onboard_ds['final_appointment_orientations'] = np.where(onboard_ds['final_appointment_orientations_y'].isna(), onboard_ds['final_appointment_orientations_x'], onboard_ds['final_appointment_orientations_y'])
#onboard_ds['final_health_plan_task'] = np.where(onboard_ds['final_health_plan_task_y'].isna(), onboard_ds['final_health_plan_task_x'], onboard_ds['final_health_plan_task_y'])

#onboard_ds.drop(columns=['final_follow_up_y', 'final_follow_up_x', 'final_appointment_orientations_y', 'final_appointment_orientations_x', 'final_health_plan_task_y', 'final_health_plan_task_x'], inplace=True)

# Export to datalek

In [17]:
di.redshift.insert_table(new_onboarding_df, table_name='caremel_onboarding', if_exists='replace')

/Users/luiz.superti/Library/Python/3.9/lib/python/site-packages/pandas/io/sql.py:1766: SAWarning: Dialect redshift:psycopg2 will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return insp.has_table(name, schema or self.meta.schema)


Table caremel_onboarding inserted successfully, with 960 rows.
